# Análisis del riesgo de incumplimiento de los prestatarios

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

# Credit Scoring y el análisis de riesgo de incumplimiento

# Contenido
* [Introducción](#intro)
* [Objetivos](#objetivos)
* [Etapas](#Etapas)
* [Descripción de los datos](#descrip)
    * [Exploración de los datos](#exploracion)
    * [Conclusiones](#conclusion1)
* [Preprocesamiento de los datos](#preprocesamiento)
    * [Transformación de los datos](#transformacion)
    * [Trabajar con valores ausentes](#ausentes)
    * [Clasificación de datos](#clasif)
    * [Conclusiones](#conclusiones_preproc)
* [Comprobación de las hipótesis](#hipotesis)
* [Conclusión general](#conclusion)



## Introducción <a id='intro'></a>
Para este proyecto, nuestro trabajo consiste en analizar a posibles factores de incumplimiento del pago de un préstamo de la división de préstamos de un banco. Las hipótesis exactas serán presentadas abajo y se requerirán aplicar técnicas de procesamiento de datos para poder demostrar de manera confiable las hipótesis requeridas. Cada espacio de código incluye sus respectivos comentarios y documentación para ayudar en la comprensión de la lógica de desarrollo del proyecto.  

## Objetivos <a id='objetivos'></a>
Probar las siguientes hipótesis:
1. Existe una relación entre tener hijos y el pago de un préstamo a tiempo.
2. Existe una relación entre el estado civil y el pago de un préstamo a tiempo.
3. Existe una relación entre el nivel de ingresos y el pago de un préstamo a tiempo.
4. Existe una relación entre los propósitos del préstamo y el pago de un préstamo a tiempo.

## Etapas <a id='etapas'></a>
Debido a que no hay ninguna información sobre la calidad de los datos ni tampoco una manera de comunicarnos con el equipo respectivo de tratamiento de datos de la institución, gran parte del tratamiento previo de los datos será realizado utilizando las suposiciones más razonables desde el punto de vista del autor. Para esto, seguiremos los siguientes pasos de manera general:
1. Descripción de los datos.
2. Preprocesamiento de los datos.
3. Prueba de las hipótesis. 

## Descripción de los datos <a id='descrip'><a/>

Empezaremos importando las librerías necesarias para el análisis y con una exploración previa de los datos.

In [1]:
import pandas as pd # importando pandas

In [2]:
credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv') # Abriendo y guardando el dataset

### Exploración de datos <a id='exploracion'><a/>

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

Teniendo como base la documentación, tenemos esta descripción de las columnas. Ahora seguimos con la primera parte del análisis exploratorio, observando las columnas y una muestra de nuestros datos:

In [3]:
credit_scoring.describe() # observando un resumen de las filas y columnas de nuestro conjunto de datos

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [4]:
credit_scoring.head(15) # mostrando las 15 primeras filas del dataset


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


En este primer vistazo, en la primera tabla observamos varios problemas. 
1. En el primer cuadro, la columna referida al número de hijos tiene valores negativos y atípicos.
2. La columna de días de empleo también tiene valores negativos y atípicos.
3. La columna de edad tiene como valor mínimo a cero.
4. En el segundo cuadro, se observan valores ausentes en dos columnas.
5. También en la columna "education" tenemos valores en minúscula y mayúscula.

Procedemos a observar todos los atributos del dataset. 

In [5]:
credit_scoring.info() # Obteniendo toda la información sobre los datos


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Tal como se sospechaba, existen dos columnas con valores distintos; por tanto, tenemos valores ausentes. Por ahora no se tiene información suficiente para determinar filas duplicadas. Analizaremos más a detalle estos valores ausentes.

In [6]:
credit_scoring[credit_scoring['days_employed'].isna()] 
#Filrando el dataframe por los valores nulos de la primera columna donde hay valores nulos


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Se puede notar que de manera extraña hay una correlación en los datos ausentes de una columna (days_employed) hacia la otra (education). Se pueden realizar muchas suposiciones al respecto y seguiremos más a fondo esta peculiaridad. Para tener una referencia, esta tabla filtrada tiene 2174 filas.

In [7]:
credit_scoring[(credit_scoring['days_employed'].isna()) & (credit_scoring['total_income'].isna())]
# Aplicando múltiples condiciones para filtrar datos en base a las columnas con valores nulos
# y analizando el número de filas y la tabla filtrada.



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Conclusión intermedia**

Se observa que esta tabla filtrada contiene a todos los valores ausentes del dataset y a la vez nos confirma la correlación de los valores ausentes en las dos columnas, ya que el número de filas respecto a la anterior tabla no ha variado. Es decir, los valores ausentes están en las mismas filas de estas dos columnas respectivas. Ahora nos faltaría determinar la naturaleza de esta extraña simetría de valores ausentes entre estas columnas.

Para continuar analizando este problema, primero observaremos el porcentaje de estos valores ausentes en relación al dataset:

Porcentaje de valores ausentes  =  # de valores ausentes  /  # total de filas en la columna respectiva
<br> Porcentaje de valores ausentes = 2174 / 21525
<br> Porcentaje de valores ausentes = 10.10 %

Es decir, tenemos un 10 % de valores ausentes en nuestras columnas respectivas. Esto es relativamente grande y existe la posibilidad de que afecte a nuestras conclusiones.

Así como existe esta peculiaridad sobre los valores ausentes, también notamos que estos valores tienen a darse en personas con "secondary education", o cuando "debt" es igual a cero. Estas suposiciones tendrían que confirmarse de la mano del equipo encargado del tratamiento de datos, pero en este caso propondremos que el factor más razonable que puede afectar a nuestros valores ausentes en "days_employed" y "total_income" es la columna "education". Pensémoslo un poco. Es posible que las personas con bajos niveles educativos hayan tenido menos oportunidades laborales e ingresos mensuales recurrentes. O que simplemente haya un pequeño porcentaje de los clientes de cada nivel educativo que prefiere no dar ese tipo de información personal, o que se haya perdido información de ese porcentaje de clientes, etc. 

Para continuar y tratar de probar estas suposiciones, en los siguientes pasos analizaremos la distribución de los valores de la columna "eduacation" con y sin valores ausentes, y así sucesivamente con las demás columnas categóricas, buscando determinar con qué columna existe una relación con los valores ausentes.

**Education**

Analizaremos la distribución de la columna que podría tener una relación teórica con nuestros valores ausentes. Veámoslo.

In [8]:
credit_scoring['education'].str.lower().value_counts(normalize=True)
# Comprobando la distribución de "education" en el conjunto de datos entero porcentualmente

secondary education    0.707689
bachelor's degree      0.244367
some college           0.034564
primary education      0.013101
graduate degree        0.000279
Name: education, dtype: float64

In [9]:
credit_scoring_na = credit_scoring.dropna(subset=['days_employed']) 
# obteniendo el dataset sin valores ausentes

In [10]:
credit_scoring_na['education'].str.lower().value_counts(normalize=True) 
# obteniendo la distribución de la columna "education" sin valores ausentes

secondary education    0.707612
bachelor's degree      0.243708
some college           0.034882
primary education      0.013488
graduate degree        0.000310
Name: education, dtype: float64

**Conclusión intermedia**

La distribución de valores prácticamente no se vio afectada al prescindir de los valores ausentes. Esto nos indicaría que quizá esta columna no afecte ni se vea afectada por los valores ausentes.

Entonces no hemos logrado determinar un patrón sobre los valores ausentes. Seguiremos intentando encontrar un posible patrón con otras columnas.


**Analizando otras posibles columnas**

Ya que aún no podemos tener una respuesta definitiva, seguiremos analizando con `debt`, `gender`, `family_status` e `income_type`.

In [11]:
credit_scoring['debt'].value_counts(normalize=True) # distribución de "debt" con valores ausentes

0    0.919117
1    0.080883
Name: debt, dtype: float64

In [12]:
credit_scoring_na['debt'].value_counts(normalize=True) # distribución de "debt" sin valores ausentes

0    0.918816
1    0.081184
Name: debt, dtype: float64

Nuevamente, esta columna no parece mostrar diferencias significativas con o sin los valores ausentes. Continuaremos con `gender`.

In [13]:
credit_scoring['gender'].value_counts(normalize=True) # distribución de "gender" con valores ausentes

F      0.661370
M      0.338583
XNA    0.000046
Name: gender, dtype: float64

In [14]:
credit_scoring_na['gender'].value_counts(normalize=True) # distribución de "gender" sin valores ausentes

F      0.658984
M      0.340964
XNA    0.000052
Name: gender, dtype: float64

Aquí existe una diferencia mínima porcentual. Estadísticamente es posible debido a que tenemos pocas categorías. Por ahora probaremos con "family_status".

In [15]:
credit_scoring['family_status'].value_counts(normalize=True) # distribución de "family_status" con valores ausentes

married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64

In [16]:
credit_scoring_na['family_status'].value_counts(normalize=True) # distribución de "family_status" sin valores ausentes

married              0.575836
civil partnership    0.193013
unmarried            0.130484
divorced             0.055966
widow / widower      0.044701
Name: family_status, dtype: float64

Esta columna no parece haberse visto afectada por los valores ausentes. Nuestra última columna sería `income_type`:

In [17]:
credit_scoring['income_type'].value_counts(normalize=True) # distribución de "income_type" con valores ausentes

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
entrepreneur                   0.000093
unemployed                     0.000093
paternity / maternity leave    0.000046
student                        0.000046
Name: income_type, dtype: float64

In [18]:
credit_scoring_na['income_type'].value_counts(normalize=True) # distribución de "income_type" sin valores ausentes

employee                       0.517493
business                       0.236525
retiree                        0.177924
civil servant                  0.067800
unemployed                     0.000103
entrepreneur                   0.000052
paternity / maternity leave    0.000052
student                        0.000052
Name: income_type, dtype: float64

Acá tampoco se observan grandes diferencias en las categorías con mayor número de valores, aunque sí en las que tienen pocos valores. Estas categorías atípicas serán tratadas posteriormente. Por ahora no hemos logrado encontrar un claro patrón de los valores ausentes hacia alguna de nuestras columnas.

**Conclusiones** <a id='conclusion1'></a>

Después de esta exploración inicial de los datos, hemos detectado varios problemas que serán tratados en los siguientes apartados. Nombraremos de manera general los problemas y cómo los abordaremos próximamente.
1. Algunas de nuestras columnas tienen problemas ortográficos, los cuales serán solucionados utilizando las transformaciones respectivas.
2. Algunas columnas presentan datos extraños y atípicos, los cuales serán corregidos mediante el análisis de la naturaleza de cada uno.
3. Analizaremos a detalle la posible ocurrencia de valores duplicados, de los cuales aún no se tienen muchos detalles.
4. Nuestros datos presentan valores ausentes que parecen mostrar un patrón aleatorio. Entonces la naturaleza de los valores ausentes es aún desconocida y debido a esta falta de información por ahora continuaremos con las correcciones y preprocesamiento de datos.

## Preprocesamiento de los datos <a id='preprocesamiento'></a>

### Transformación de datos <a id='transformacion'></a>

Teníamos la siguiente información de cada columna después del análisis descriptivo y exploratorio:
* `children` tenía datos extraños y atípicos.
* `days_employed` tenía datos extraños, atípicos y ausentes.
* `dob_years` tenía datos extraños.
* `education` tenía problemas ortográficos.
* `family_status` no parecía mostrar problemas.
* `gender` no parecía mostrar problemas.
* `income_type` no parecía mostrar problemas.
* `debt` no parecía mostrar problemas.
* `total_income` tenía datos ausentes.
* `purpose` no parecía mostrar problemas.

Empezaremos tratando a la columna referida a educación y sus valores:


In [19]:
print(credit_scoring['education'].unique()) # Obteniendo los valores únicos de educación

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


Tal como se había observado, tiene problemas ortográficos. Corrigiéndolos:

In [20]:
credit_scoring['education'] = credit_scoring['education'].str.lower() # Arreglando los registros

In [21]:
print(credit_scoring['education'].unique()) # Comprobando los valores únicos de educación


["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


Una vez solucionado, pasamos a la columna referida a la cantidad de niños:

In [22]:
print(credit_scoring['children'].value_counts()) # Obteniendo la distribución de valores
credit_scoring['children'].value_counts(normalize=True) # Obteniendo la distribución porcentual

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64

Observando la distribución de los valores de la columna "children" confirmamos que tenemos valores extraños y atípicos. ¿Qué tan probable es que una familia tenga 20 hijos? ¿Y es posible que hayan '-1' hijos? Este tipo de errores deberían ser comunicados y solucionados con el área respectiva, pero no es posible ahora. Entonces plantearemos algunas hipótesis razonables sobre la naturaleza de estos errores:

- Si se tratara de un error humano, es posible que el '20' haya sido en realidad un '2', mientras que para el '-1' se podrían barajar más posibilidades: es posible que el que introdujo los datos haya puesto simplemente una raya '-' para registrar familias donde no se pudo obtener esa información o que quizá no tienen hijos. También es posible que en realidad sea un '1' y que la raya sea un artefacto en el proceso de introducir los datos a la computadora.
- Si se tratara de un error computacional o técnico, sigue siendo bastante probable que el '20' en realidad sea un '2', mientras que para el '-1' se plantearía que podría ser un error computacional, una mala lectura de los datos o simplemente valores nulos que la computadora no supo procesar.

Para ayudarnos en el análisis la distribución de forma porcentual es presentada también, debajo. Se observa que los datos problemáticos son de alrededor del 0.57 % de la columna.

Personalmente tiene más sentido que sea un error humano, siendo así el 20 un 2 y el -1 en realidad un 1. Viendo los porcentajes también notamos que incluso incrementando esos respectivos valores, los cambios porcentuales serían mínimos. Entonces procedemos a corregir estos datos.

In [23]:
credit_scoring['children'].replace(-1, 1, inplace=True) # Reemplazando -1 por 1
credit_scoring['children'].replace(20, 2, inplace=True) # Reemplazando 20 por 2

In [24]:
credit_scoring['children'].value_counts() # Comprobando que esté solucionado

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Una vez solucionada esta columna, continuamos con `days_employed`. Si recordamos, esta columna tenía problemas con valores extraños, atípicos y ausentes. Primero abordaremos los valores atípicos y extraños.

In [25]:
print(credit_scoring['days_employed'].describe()) # Volviendo a obtener la información de esta columna
print()
print(f"Datos negativos: {(credit_scoring['days_employed'] < 0).sum()}") 
# Obteniendo la cantidad de números negativos y mostrándolos
print(f"Datos positivos: {(credit_scoring['days_employed'] > 0).sum()}")
# Obteniendo la cantidad de números positivos y mostrándolos        

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Datos negativos: 15906
Datos positivos: 3445


Resaltamos un par de cuestiones:
* Notamos que la mayor parte de los valores son negativos, observando la distribución y el conteo realizado.
* Dada la escala del problema, no se pueden eliminar estos errores.
* Ya que no se puede obtener más información acerca de este error, asumiremos que en este caso se trata de un error técnico, ya que es poco probable que una persona introduzca tantos valores errados.

Para este primer caso, la solución será corregir el signo en los valores negativos:


In [26]:
credit_scoring['days_employed'] = credit_scoring['days_employed'].abs()
# Corrigiendo los valores negativos

In [27]:
credit_scoring['days_employed'].describe() 
# Comprobando el resultado y analizando el siguiente problema

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Ahora notamos un par de cosas: la desviación estándar es muy grande, la media y mediana están muy alejadas entre sí y el valor máximo es totalmente irreal: ¡400 mil días son más de mil años!
Para abordar este problema, primero veremos cuándos valores son los que están desplazando nuestra distribución. Tomando en cuenta que la edad máxima de los clientes es de 75 años, asumiremos el límite máximo de 20 mil horas trabajadas en total.

In [28]:
len(credit_scoring[credit_scoring['days_employed'] > 20000]) # Contando el número de filas

3445

En realidad, me gustaría aclarar que hice un análisis gráfico de la distribución de los valores y encontramos dos distribuciones:
* La primera está tiene un máximo de alrededor de 18 mil días.
* La segunda empieza desde los 300 mil aprox. en adelante y tiene los 3445 valores.

Es evidente que estos 3445 valores atípicos no pueden ser reales y en realidad se deban a algún tipo de error. Determinar el origen de este error requeriría un análisis conjunto con el equipo respectivo, pero como no es factible en este caso y al no haber más información al respecto, convertiremos estos datos en "Nan" para luego estandarizarlos al completar los datos ausentes totales.  

In [29]:
credit_scoring['days_employed'] = credit_scoring['days_employed'].apply(lambda x: None if x>20000 else x)
# Utilizando otra función lambda para corregir este problema

In [31]:
len(credit_scoring[credit_scoring['days_employed'] > 20000]) # Comprobando la corrección

0

In [32]:
credit_scoring['days_employed'].value_counts(dropna=False).head() 
# analizando los nuevos datos ausentes, que se incrementaron

NaN            5619
144.185854        1
2358.122341       1
2569.204627       1
3545.955468       1
Name: days_employed, dtype: int64

In [33]:
credit_scoring['days_employed'].describe() # Observando sus estadísticos nuevamente

count    15906.000000
mean      2353.015932
std       2304.243851
min         24.141633
25%        756.371964
50%       1630.019381
75%       3157.480084
max      18388.949901
Name: days_employed, dtype: float64

Ahora observamos que los problemas en esta columna están "corregidos", faltanto nuestros valores ausentes. La desviación estándar es mucho más pequeña y la media y mediana no están tan alejadas entre sí. Ahora continuaremos con `dob_years`.

In [34]:
print(credit_scoring['dob_years'].describe()) # Obteniendo sus estadísticos
print()
zero_dob_years = credit_scoring['dob_years'].value_counts()[0] 
# Contando los valores de edad y la cantidad del valor atípico cero
print(f"Cantidad de valores iguales a cero: {zero_dob_years}")
# Mostrando la cantidad de ceros con formato

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Cantidad de valores iguales a cero: 101


Del análisis descriptivo, teníamos que existía un valor mínimo cero. Ahora también tenemos la cantidad de valores de esta columna, que son 101. Es evidente que cero no es una edad real de alguien que solicita un préstamo en el banco. De hecho, en la mayoría de países, el mínimo legal es de 18 años, valor que extrañamente no aparece en esta distribución a pesar de tener una muestra relativamente grande.

Debido a que no podemos obtener más información al respecto, lo siguiente a realizar es obtener la distribución sin el valor atípico y de ahí obtener la mediana para completar con ese valor exacto, de esta manera no afectaremos a la distribución de la columna.

In [35]:
credit_sc_dob = credit_scoring[credit_scoring['dob_years'] != 0] # obteniendo el dataset sin el valor atípico
credit_sc_dob['dob_years'].describe() # obteniendo los estadísticos y la nueva mediana

count    21424.000000
mean        43.497479
std         12.246934
min         19.000000
25%         33.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

De esta distribución, tenemos que la mediana a utilizar es 43, valor que varía respecto a la distribución con el valor atípico. Este es un pequeño detalle a tener en cuenta en proximos análisis.

Procedemos a realizar el cambio:

In [36]:
credit_scoring['dob_years'].replace(0, 43, inplace=True) # Reemplazando 0 por 18

In [37]:
credit_scoring['dob_years'].describe() # Comprobando si el problema está resuelto

count    21525.000000
mean        43.495145
std         12.218213
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Después de corregir los valores de `dob_years`, ahora nos enfocaremos en `family_status`. Recordemos que esta columna no parecía mostrar problemas.

In [38]:
credit_scoring['family_status'].value_counts() # Observando el conteo de valores

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

La clasificación en esta columna y sus valores parecen no tener problemas que solucionar. Avancemos a la columna `gender`.


In [39]:
credit_scoring['gender'].value_counts() # Veamos los valores en la columna del género

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

"XNA" es un valor totalmente extraño y sin posible interpretación. Lo más cercano es que se haya querido un "NA". Al ser un valor único, me parece que la mejor solución es simplemente prescindir de esta fila.

In [40]:
credit_scoring = credit_scoring[credit_scoring['gender'] != 'XNA'] # Eliminando 'XNA' del dataset

In [41]:
credit_scoring['gender'].value_counts() # Comprobando si está solucionado

F    14236
M     7288
Name: gender, dtype: int64

Tal parece que ya hemos solucionado esa columna. Continuamos con `income_type`.

In [42]:
credit_scoring['income_type'].value_counts(dropna=False) 
# Obteniendo el conteo o distribución de valores

employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

A simple vista, estos valores podrían no tener problemas. Pero analizando un poco, hay categorías que tienen pocos datos, tienen comportamiento de datos atípicos, muy alejados de la distribución de datos. 

Para apoyar esta suposición, también tenemos que, en términos económicos, tener como fuente de ingreso a 'business' o 'entrepreneur' podría significar lo mismo; 'student' no es una categoría de ingresos como tal, es bastante probable que la persona que se identificó como estudiante no esté empleada; y el 'paternity/maternity leave' (baja por paternidad o maternidad) se da en personas empleadas que dejan de trabajar temporalmente, pero siguen teniendo el vínculo laboral y en la mayoría de los casos aún perciben sus ingresos normales como empleados.

Dada esta explicación, agruparemos estos datos atípicos de la siguiente manera:
- Los valores asignados a 'entrepreneur' serán asignados a 'business'.
- Los de 'paternity/maternity leave' serán clasificados como 'employee'.
- Los de 'student' serán asignados a 'unemployed'.

In [43]:
credit_scoring.loc[credit_scoring['income_type'] == 'entrepreneur', 'income_type'] = 'business'
# Asignando 'business' a 'entrepeneur'
credit_scoring.loc[credit_scoring['income_type'] == 'paternity / maternity leave', 'income_type'] = 'employee'
# Asignando 'employee' a 'paternity/maternity leave'
credit_scoring.loc[credit_scoring['income_type'] == 'student', 'income_type'] = 'unemployed'
# Asignando 'unemployed' a 'student'

In [44]:
credit_scoring['income_type'].value_counts(dropna=False) # Comprobando el resultado

employee         11120
business          5086
retiree           3856
civil servant     1459
unemployed           3
Name: income_type, dtype: int64

Una vez solucionado, recordemos que nuestra columna `total_income` necesitaba un tratamiento para sus valores ausentes. Esto será tratado en el siguiente apartado. Para el caso de `purpose`, mostraremos sus valores únicos para darnos una idea de sus problemas.

In [45]:
credit_scoring['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Aunque parecía que no se tenían problemas en esta columna, la verdad es que las categorías son redundantes. Este trabajo será analizado en un próximo apartado dedicado solo a solucionar este problema. Por ahora continuemos con los datos duplicados y si existen en nuestro dataset.

In [46]:
credit_scoring.duplicated().sum() # Contando las filas duplicadas

71

Tal parece que tenemos 71 filas duplicadas. Si lo pensamos un poco, podríamos suponer que no tenemos la suficiente información para determinar que las filas son duplicadas o no. Pero esta idea se descarta al tener en cuenta que tenemos variables flotantes, cuyos valores son extremadamente difíciles de que coincidan a la vez, al menos en teoría. Por tanto, ahora nos encargaremos de estos duplicados.

In [47]:
credit_scoring = credit_scoring.drop_duplicates().reset_index(drop=True) 
# Eliminando los duplicados y restaurando los índices

In [48]:
credit_scoring.duplicated().sum() # Última comprobación para ver si tenemos duplicados


0

In [49]:
credit_scoring.info() 
# Obteniendo los nuevos metadatos, después de haber ejecutado estas primeras manipulaciones

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     15905 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      19350 non-null  float64
 11  purpose           21453 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Luego de realizar todos estos cambios, tratando de preservar el mayor número de datos posibles, tenemos un nuevo dataframe con una pequeña diferencia en el total de filas: son 21452 en lugar de los 21525 originales. Se puede considerar que, en términos porcentuales, no se han perdido muchos datos  ni información que pueda afectar a nuestras conclusiones (alrededor del 0.33 % de los datos originales han sido eliminados). 

Nuestro siguiente objetivo será abordar los valores ausentes.

### Trabajando con valores ausentes <a id='ausentes'></a>

Para comenzar, quizá sea necesario obtener los diccionarios de las columnas categóricas del dataset donde hay una relación id-'nombre'. Se irán agregando diccionarios extra si fuesen necesarios en el futuro. Es posible que estas relaciones nos sirvan o hagan más fáciles algunas labores posteriores.

In [50]:
dic_category_education = credit_scoring[['education_id', 'education']] # Obteniendo las columnas
dic_category_education = dic_category_education.drop_duplicates().reset_index(drop=True) # Obteniendo sus valores respectivos
print(dic_category_education) # Mostrando los valores
print()
dic_category_family = credit_scoring[['family_status_id', 'family_status']] # Obteniendo las columnas
dic_category_family = dic_category_family.drop_duplicates().reset_index(drop=True) # Obteniendo sus valores respectivos
print(dic_category_family) # Mostrando los valores


   education_id            education
0             0    bachelor's degree
1             1  secondary education
2             2         some college
3             3    primary education
4             4      graduate degree

   family_status_id      family_status
0                 0            married
1                 1  civil partnership
2                 2    widow / widower
3                 3           divorced
4                 4          unmarried


#### Restaurando valores ausentes en `total_income`

Como ya se había detallado, tenemos dos columnas con valores ausentes: "days_employed" y "total_income". Se realizará un análisis de acuerdo a las características de cada variable para completar estos valores ausentes. Empezaremos con "total_income", una de las variables más importantes de nuestro análisis.

Empezaremos creando una categoría de edad que luego añadiremos a una nueva columna. Esta categoría es posible que nos ayude a calcular la distribución de los ingresos para completar los valores ausentes. La categorización que me parece adecuada para este tipo de análisis referido a temas financieros es la siguiente:
- young: de 18 a 25 años.
- young adult: de 26 a 35 años.
- intermediate adult: de 36 a 50 años.
- advanced adult: de 51 a 64 años.
- retired: de 65 en adelante.

Esta clasificación es teniendo en cuenta las edades, ingresos respectivos y el empleo que tienen o del cual llegan a retirarse.

Definiremos la función que nos ayude a clasificar edades:

In [51]:
def age_group(age): # la función nos devuelve los valores presentados para cada rango de edad
    if age < 18 or pd.isna(age): # si encuentra valores extraños o nulos, devuelve la cadena 'NA'
        return 'NA'
    elif age <= 25:
        return 'young'
    elif age <= 35:
        return 'young adult'
    elif age <= 50:
        return 'intermediate adult'
    elif age <= 64:
        return 'advanced adult'
    return 'retired'


    

In [52]:
print(age_group(15)) # probando la función
print(age_group(25)) # probando la función
print(age_group(30)) # ...
print(age_group(38))
print(age_group(55)) # ...
print(age_group(65)) # probando la función



NA
young
young adult
intermediate adult
advanced adult
retired


In [53]:
credit_scoring['age_category'] = credit_scoring['dob_years'].apply(age_group)
# Creando una nueva columna basada en la función



In [54]:
credit_scoring['age_category'].head(10) 
# Comprobando los valores en la nueva columna



0    intermediate adult
1    intermediate adult
2           young adult
3           young adult
4        advanced adult
5           young adult
6    intermediate adult
7    intermediate adult
8           young adult
9    intermediate adult
Name: age_category, dtype: object

Generalmente, los ingresos de las personas dependen del nivel educativo, la experiencia laboral, el tipo de empleo y la edad. Tenemos toda esta información en nuestro dataset, así que trataremos de demostrar cuán importantes son estos factores en esta muestra, analizando sus respectivas distribuciones.

Para poder completar estos valores ausentes, analizaremos el dataset prescindiendo de ellos. Esta tabla nos permitirá obtener los datos necesarios para completar el dataset original.

In [55]:
credit_scoring_without_na = credit_scoring.dropna(subset=['total_income']) 
# Guardando el nuevo datset sin valores ausentes en "total_income"
credit_scoring_without_na.head(10)
# Observando y comprobando el nuevo dataset

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,intermediate adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,intermediate adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,young adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,young adult
4,0,NaN,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,advanced adult
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,young adult
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,intermediate adult
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,intermediate adult
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,young adult
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,intermediate adult


In [56]:
income_grouped_mean_1 = credit_scoring.pivot_table(index='education', values='total_income', aggfunc='mean')
income_grouped_mean_1
# Examinando los valores medios de los ingresos en función del nivel educativo

,total_income
education,
bachelor's degree,33142.802434
graduate degree,27960.024667
primary education,21144.882211
secondary education,24594.503037
some college,29040.132990


In [57]:
income_grouped_mean_2 = credit_scoring.pivot_table(index='income_type', values='total_income', aggfunc='mean')
income_grouped_mean_2
# Examinando los valores medios de los ingresos en función del tipo de empleo

,total_income
income_type,
business,32397.115286
civil servant,27343.729582
employee,25819.123443
retiree,21940.394503
unemployed,19246.993667


In [58]:
income_grouped_mean_3 = credit_scoring.pivot_table(index='age_category', values='total_income', aggfunc='mean')
income_grouped_mean_3
# Examinando los valores medios de los ingresos en función de la categoría de edades creada

,total_income
age_category,
advanced adult,25206.111775
intermediate adult,28422.717718
retired,21542.650450
young,23447.795802
young adult,27613.702541


In [59]:
income_grouped_mean_4 = credit_scoring_without_na[['total_income', 'days_employed']].describe()
income_grouped_mean_4
# Examinando la distribución de los ingresos con los días de empleo a la vez

,total_income,days_employed
count,19350.000000,15905.000000
mean,26787.266688,2353.015581
std,16475.822926,2304.316291
min,3306.762000,24.141633
25%,16486.515250,756.281915
50%,23201.873500,1629.997862
75%,32547.910750,3157.654315
max,362496.645000,18388.949901


In [60]:
inc_gr_median_1 = credit_scoring.pivot_table(index='education', values='total_income', aggfunc='median')
inc_gr_median_1
# Examinando los valores medianos de los ingresos en función del factor educativo


,total_income
education,
bachelor's degree,28054.5310
graduate degree,25161.5835
primary education,18741.9760
secondary education,21836.5830
some college,25608.7945


In [61]:
income_grouped_median_2 = credit_scoring.pivot_table(index='income_type', values='total_income', aggfunc='median')
income_grouped_median_2
# Examinando los valores medianos de los ingresos en función del tipo de ingreso

,total_income
income_type,
business,27577.2720
civil servant,24071.6695
employee,22814.0140
retiree,18962.3180
unemployed,15712.2600


In [62]:
income_grouped_median_3 = credit_scoring.pivot_table(index='age_category', values='total_income', aggfunc='median')
income_grouped_median_3
# Examinando los valores medianos de los ingresos en función de la categoría de edad creada

,total_income
age_category,
advanced adult,21738.7255
intermediate adult,24691.2115
retired,18471.3910
young,21423.8355
young adult,24193.5905


Analizando los ingresos en función de cada factor posible, parece razonable suponer que en general todas son buenas aproximaciones, ya que los valores de cada categoría se parecen en medias y medianas. Es más, en cierta medida siguen el patrón lógico que se esperaría, aunque con algunas excepciones. Teniendo en cuenta algunas guías teóricas sobre qué factor es el más importante, usaremos como factor principal al educativo. 

Es sabido en la literatura que una variable referida a ingresos tiene muchos valores outliers, en este caso no es la excepción: la media y mediana están un poco desplazadas. Debido a esto, decidiremos usar la mediana para completar los valores ausentes.

Crearemos una función para completar los valores ausentes en base a las medianas de cada nivel educativo:

In [63]:
def complete_nan(data): # Definiendo la función
    education = data['education']
    income = data['total_income']
    if pd.isna(income): # Si el valor de "income" es ausente, devuelve el valor respectivo
        try:
            return inc_gr_median_1.loc[education, 'total_income']
        except:
            return 'NA'  # Si existe un valor educativo distinto, devuelve "NA"
    return income   # Si el valor de "income" no es nulo nos devuelve el mismo valor
#  función que usaremos para completar los valores ausentes
        
        

In [64]:
data_values = {'education': ['primary education', 'graduate degree', 'some college', "bachelor's degree", 'secondary education'],
              'total_income': [None, 10000, None, 50000, None]} # Creando el dataframe de prueba
data_test = pd.DataFrame(data_values) # Creando el dataframe de prueba
data_test['total_income'] = data_test.apply(complete_nan, axis=1) # Comprobando si funciona
data_test

,education,total_income
0,primary education,18741.9760
1,graduate degree,10000.0000
2,some college,25608.7945
3,bachelor's degree,50000.0000
4,secondary education,21836.5830


In [65]:
credit_scoring['total_income'] = credit_scoring.apply(complete_nan, axis=1)
# Aplicando la función a cada fila respectiva


In [66]:
print(credit_scoring[credit_scoring['total_income'].isna()]) # Comprobando si aún hay valores ausentes
credit_scoring.head(10) # Mostrando el dataset con los datos rellenados


Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, age_category]
Index: []


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,intermediate adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,intermediate adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,young adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,young adult
4,0,NaN,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,advanced adult
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,young adult
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,intermediate adult
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,intermediate adult
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,young adult
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,intermediate adult


Ahora comprobaremos si esta columna ha sido corregida analizando sus atributos:

In [67]:
credit_scoring.info() # Comprobar el número de entradas en la columna

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     15905 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  age_category      21453 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


####  Restaurar valores en `days_employed`

Para empezar con el análisis, debemos pensar en cuáles podrían ser los factores que nos puedan ayudar a completar los valores ausentes de los días de empleo del cliente. La deducción es sencilla: la cantidad de días en los que el cliente ha trabajado depende directamente de su edad, y en mucha menor medida, del tipo de empleo y el nivel educativo. En este caso, solo analizaremos la relación entre los días empleados y la edad para ayudarnos a completar sus valores ausentes de la mejor forma posible, ya se utilizando la media o mediana. También tengamos en cuenta que esta no es más que una conjetura basada en la lógica. La verdadera naturaleza de los valores ausentes también debería ser analizada y determinada con el equipo respectivo.

In [68]:
emp_gr_median = credit_scoring.pivot_table(index='age_category', values='days_employed', aggfunc='median')
emp_gr_median
# Examinando los valores medianos de los días de empleo en función del tipo de la categoría de edad

,days_employed
age_category,
advanced adult,2319.817259
intermediate adult,1958.054821
retired,2876.221697
young,796.983636
young adult,1343.205241


In [69]:
employed_grouped_mean = credit_scoring.pivot_table(index='age_category', values='days_employed', aggfunc='mean')
employed_grouped_mean
# Examinando los valores medios de los días de empleo en función del tipo de la categoría de edad

,days_employed
age_category,
advanced adult,3362.724643
intermediate adult,2624.713674
retired,4010.478357
young,923.122928
young adult,1681.227071


Luego de observar que los valores no son tan parecidos entre sí en algunos casos, sospechamos que se debe a la existencia de algunos valores atípicos que aún persisten en la distribución de `days_employed`. Para evitar llevar este sesgo al momento de completar los ausentes, utilizaremos nuevamente la mediana. Para esto, guardaremos los valores de las medianas y luego crearemos una función que nos ayude a rellenar los ausentes:

In [70]:
# Escribamos una función que calcule medianas según el parámetro de edad:
def fix_na_days(data): # Definiendo la función
    days = data['days_employed']
    age = data['age_category']
    if pd.isna(days): # si el valor de "days_employed" es ausente, devuelve el valor respectivo
        try:
            return emp_gr_median.loc[age, 'days_employed']
        except:
            return 'NA' # si hay una categoría fuera de estas edades, devuelve "NA"
    return days # si "days_employed" no es nulo, nos devuelve el mismo valor


In [71]:
data_val = {'age_category': ['retired', 'young', 'intermediate adult', "young adult", 'advanced adult'],
              'days_employed': [None, 1500, None, 600, None]} # Creando el dataframe de prueba
data_test1 = pd.DataFrame(data_val) # Creando el dataframe de prueba
data_test1['days_employed'] = data_test1.apply(fix_na_days, axis=1) # Comprobando si funciona
data_test1
# Comprobando que la función funciona

,age_category,days_employed
0,retired,2876.221697
1,young,1500.000000
2,intermediate adult,1958.054821
3,young adult,600.000000
4,advanced adult,2319.817259


In [72]:
credit_scoring['days_employed'] = credit_scoring.apply(fix_na_days, axis=1) 
# Aplicando la función al days_employed

In [73]:
print(credit_scoring[credit_scoring['days_employed'].isna()]) # Comprobando si aún hay valores ausentes
credit_scoring.head(10) # Mostrando el dataset con los datos rellenados

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, age_category]
Index: []


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,intermediate adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,intermediate adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,young adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,young adult
4,0,2319.817259,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,advanced adult
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,young adult
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,intermediate adult
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,intermediate adult
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,young adult
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,intermediate adult


Ahora observaremos los atributos del dataframe para comparar nuestra columna rellenada y realizar nuevamente una verficación:

In [74]:
credit_scoring.info() # Analizando si ya el problema está resuelto

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  age_category      21453 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Como se observa, todas nuestras columnas tienen datos completos: pudimos deshacernos de los valores ausentes. Ahora continuaremos con las columnas que aún presentan problemas o necesitan correciones.

### Clasificación de datos <a id='clasif'></a>

Habíamos dejado pendiente clasificar nuestros datos en la columna `purpose`. Agrupar y clasificar esta columna será de utilidad para probar una de nuestras hipótesis. Hay que tener en mente que esta columna contiene texto y su manejo será un poco distinto al de la columna siguiente a analizar.

También será necesario clasificar nuestra columna `total_income`, debido a que el análisis con la columna en bruto será muy complicado utilizando valores flotantes y muy diversos. En este caso, seguiremos un proceso similar a la clasificación de edades realizada líneas arriba.

Dicho esto, empezaremos analizando nuevamente a estas columnas:

In [75]:
credit_scoring[['purpose', 'total_income']].head(10) # mostrando las columnas 

,purpose,total_income
0,purchase of the house,40620.102
1,car purchase,17932.802
2,purchase of the house,23341.752
3,supplementary education,42820.568
4,to have a wedding,25378.572
5,purchase of the house,40922.170
6,housing transactions,38484.156
7,education,21731.829
8,having a wedding,15337.093
9,purchase of the house for my family,23108.150


Empezaremos con `purpose`, observando sus valores únicos:

In [76]:
credit_scoring['purpose'].unique()
# Comprobar los valores únicos

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Podemos notar temáticas similares con las cuales podemos agrupar y clasificar estos valores, debido a que son redundantes y entorpecería nuestro análisis. Las temáticas o grupos principales identificados son:
* house purchase
* car purchase
* education
* wedding
* real estate
* property

En base a estas temáticas, crearemos una función que detecte las cadenas respectivas para clasificar los datos, de la siguiente manera:

In [77]:
def purpose_group(row): # la función busca una subcadena referida al tema y lo agrupa en la categoría respectiva
    if 'hous' in row:
        return 'house purchase'
    if 'car' in row:
        return 'car purchase'
    if 'educat' in row or 'university' in row: # la subcadena 'university' también se categoriza en 'education'
        return 'education'
    if 'wedding' in row:
        return 'wedding'
    if 'estate' in row:
        return 'real estate'
    if 'property' in row:
        return 'property'
    return 'NA' # si no hay subcadenas reconocibles, devuelve 'NA'

In [78]:
credit_scoring['purpose_category'] = credit_scoring['purpose'].apply(purpose_group) 
# creando la nueva columna con las categorías
print(len(credit_scoring['purpose_category']))
# obteniendo el tamaño para verificar que no falten valores
credit_scoring['purpose_category'].unique()
# imprimiendo los valores únicos para verficar

21453


array(['house purchase', 'car purchase', 'education', 'wedding',
       'real estate', 'property'], dtype=object)

Tal parece que nuestra función logró el objetivo. Para cerciorarnos una vez más veremos un ejemplo de las columnas en cuestión:

In [79]:
credit_scoring[['purpose','purpose_category']].head(10) # una última verificación

,purpose,purpose_category
0,purchase of the house,house purchase
1,car purchase,car purchase
2,purchase of the house,house purchase
3,supplementary education,education
4,to have a wedding,wedding
5,purchase of the house,house purchase
6,housing transactions,house purchase
7,education,education
8,having a wedding,wedding
9,purchase of the house for my family,house purchase


Una vez clasificado y solucionada esta columna, también será necesario analizar la columna `total_income`, por las razones expuestas al empezar este capítulo, que es básicamente la dificultad para analizar los diversos valores que tiene esta columna por su propia naturaleza que se muestra ahora: 

In [80]:
credit_scoring['total_income']
# Revisando todos los datos numéricos en la columna

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21448    35966.698
21449    24959.969
21450    14347.610
21451    39054.888
21452    13127.587
Name: total_income, Length: 21453, dtype: float64

In [81]:
credit_scoring['total_income'].describe()
# Obteniendo estadísticas resumidas para la columna

count     21453.000000
mean      26465.838089
std       15701.456362
min        3306.762000
25%       17219.352000
50%       22582.311000
75%       31327.922000
max      362496.645000
Name: total_income, dtype: float64

Estos estadísticos nos ayudarán a agrupar a los clientes en base a sus ingresos mensuales (`total_income`). Para tener una distribución de valores más o menos homogénea, usaremos el percentil 25 y 75 como límites, y luego más de 50 mil para valores atípicos que se suelen presentar en las distribuciones de ingresos.

Es decir, de la siguiente forma:
* low income: `total_income`< 17219.352
* middle income: 17219.352 <= `total_income` < 31327.922
* high income: 31327.922 <= `total_income` < 50000
* very high income: `total_income` >= 50000


In [82]:
def income_grouped(row): # definimos la función
    if row < 0:  # si se introduce algún valor negativo, se devuelve 'NA'
        return 'NA'
    elif row < 17219.352:
        return 'low income'
    elif row < 31327.922:
        return 'middle income'
    elif row < 50000:
        return 'high income'
    return 'very high income' # para los valores fuera de este rango, devuelve 'very high income'

# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos



In [83]:
credit_scoring['income_level'] = credit_scoring['total_income'].apply(income_grouped)
credit_scoring[['total_income','income_level']].head()
# Crear una columna con categorías


,total_income,income_level
0,40620.102,high income
1,17932.802,middle income
2,23341.752,middle income
3,42820.568,high income
4,25378.572,middle income


In [84]:
credit_scoring.pivot_table(index='income_level', values='total_income', aggfunc='count')
# contando los valores de cada categoría para ver la distribución

,total_income
income_level,
high income,4044
low income,5363
middle income,10726
very high income,1320


Como se observa, la distribución de valores tiende a parecerse a una distribución de ingresos promedio de un país, ciudad o muestras análogas: una cantidad para ingresos bajos (grupo de ingresos bajos), la mayor parte agrupada de personas con ingresos medios (clase media), un cantidad menor para los altos ingresos y otra mucho menor para los ingresos muy altos. 

### Conclusiones <a id='conclusiones_preproc'></a>

En esta etapa del preprocesamiento, nos encargamos de todos los problemas relevantes para poder analizar nuestras variables de forma que nuestras conclusiones sea las mejores posibles.

* Corregimos los errores ortográficos de algunas columnas.
* Nos encargamos de corregir algunos valores atípicos y extraños de otras columnas.
* También nos encargamos de los datos duplicados.
* Creamos algunas funciones y agrupaciones para clasificar mejor y completar los valores ausentes de la mejor forma posible.

Ahora que tenemos nuestro dataset preprocesado, pasamos a la última etapa del proyecto: probar las hipótesis.

## Comprobación de las hipótesis <a id='hipotesis'></a>


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [85]:
test_hyp_1 = credit_scoring.pivot_table(index='children', columns='debt', values='dob_years', aggfunc='count', margins=True)
# generando el conteo de valores sobre hijos y tasa de incumplimiento utilizando una tabla dinámica y guardándola
print(test_hyp_1)
# mostrando la tabla pivote
table_hyp_1 = test_hyp_1[1]/test_hyp_1['All']
# generando los porcentajes
table_hyp_1
# comprobando la serie final con las tasas de incumplimiento en función del número de hijos

debt            0       1    All
children                        
0         13027.0  1063.0  14090
1          4410.0   445.0   4855
2          1926.0   202.0   2128
3           303.0    27.0    330
4            37.0     4.0     41
5             9.0     NaN      9
All       19712.0  1741.0  21453


children
0      0.075444
1      0.091658
2      0.094925
3      0.081818
4      0.097561
5           NaN
All    0.081154
dtype: float64

**Conclusión**

Porcentualmente, tomaremos como referencia el porcentaje global o promedio, que es de 8.11 %.
* De manera general, la hipotesis podría ser correcta, ya que los porcentajes de personas con hijos es mayor en todos los casos respecto (todos mayores a 8 %) a los que no tienen ninguno (7.54 %). Esta tendencia solo se ve contrastada por las personas con cinco hijos, de los cuales parece que, en esta muestra, ninguno de ellos tuvo un impago.
* Si analizamos la tendencia, es decir qué ocurre a medida que una persona tiene más hijos o si es más o menos probable que a más hijos ocurra algo con los impagos, esto no se podría tener claro realmente. Si bien es cierto que existe una brecha entre los que tienen hijos y los que no, la lógica nos haría suponer que a más hijos es más probable un impago, pero en esta muestra, las personas con tres y cinco hijos quiebran esta lógica.

Para cada análisis, debemos recordar que se trata de una muestra o datos proporcionados por la institución, así que nuestras conclusiones deben estar enmarcadas en ese contexto: solo funcionan para esta muestra. No se puede generalizar sin pruebas estadísticas más rigurosas.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [87]:
test_hyp_2 = credit_scoring.pivot_table(index='family_status', columns='debt', values='dob_years', aggfunc='count', margins=True)
# generando el conteo de valores sobre el estado familiar y tasa de incumplimiento utilizando una tabla dinámica y guardándola
print(test_hyp_2)
# mostrando la tabla pivote
table_hyp_2 = test_hyp_2[1] / test_hyp_2['All']
# generando los porcentajes
table_hyp_2
# comprobando la serie final con las tasas de incumplimiento en función del estado familiar

debt                   0     1    All
family_status                        
civil partnership   3762   388   4150
divorced            1110    85   1195
married            11408   931  12339
unmarried           2536   274   2810
widow / widower      896    63    959
All                19712  1741  21453


family_status
civil partnership    0.093494
divorced             0.071130
married              0.075452
unmarried            0.097509
widow / widower      0.065693
All                  0.081154
dtype: float64

**Conclusión**

Nuevamente tomaremos como referencia al promedio global de 8.11 %.
* La categoría con menor tasa de impago es la de "viudo/viuda" con un 6.56 % de proporción de personas que alguna vez incumplieron un préstamo. 
* La categoría con mayor tasa de impago es la de "soltero" (unmarried) con un 9.75 % de tasa. Es decir, es ligeramente más probable que una persona soltera no tenga sus pagos a tiempo.
* Existe una controversia entre "unión civil" (9.35 %) y "casado(a) (7.54 %)". En teoría, deberían tener comportamientos parecidos. Habría que analizar a qué clase de uniones se les considera "civil partnership" en el país respectivo y por qué esta tiende a tener más clientes riesgosos respecto a su análogo de "married".
* Finalmente, podríamos decir que existen diferencias de la tasa de impago para cada categoría respecto a la media y entre categorías, confirmando nuestra hipótesis de que el estado civil o familiar podría influir en la tendencia al pago puntual. 

Como siempre, debemos recordar que estas conclusiones se basan y funcionan solo en esta muestra.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [88]:
test_hyp_3 = credit_scoring.pivot_table(index='income_level', columns='debt', values='dob_years', aggfunc='count', margins=True)
# generando el conteo de valores sobre el la categoría de ingresos y tasa de incumplimiento utilizando una tabla dinámica y guardándola
print(test_hyp_3)
# mostrando la tabla pivote
table_hyp_3 = test_hyp_3[1] / test_hyp_3['All']
# generando los porcentajes
table_hyp_3
# comprobando la serie final con las tasas de incumplimiento en función del nivel de ingresos

debt                  0     1    All
income_level                        
high income        3753   291   4044
low income         4936   427   5363
middle income      9795   931  10726
very high income   1228    92   1320
All               19712  1741  21453


income_level
high income         0.071958
low income          0.079620
middle income       0.086798
very high income    0.069697
All                 0.081154
dtype: float64

**Conclusión**

Recordemos nuestra referencia al promedio global de 8.11 %.
* La categoría con menor tasa de incumplimiento es la de "very high income" con 6.97 %. Personas con estos niveles de ingresos tienen ligeramente menos probabilidades de tener un impago.
* La categoría con mayor tasa de incumplimiento es la de "middle income" con un 8.68 %, ligeramente superior a la media. Es posible que se trate de un artefacto estadístico debido a la metodología utilizada para clasificar y agrupar a las personas, que podría intoducir ciertos sesgos o insignificancias estadísticas. Pero siguiendo la información actual, es ligeramente más probable que una persona de ingresos medios tenga problemas de incumplimiento.
* Los demás valores se mantienen debajo o cerca a la media, así que estas categorías en general no afectan de manera significativa a la tasa de incumplimiento.
* Finalmente, no es clara una tendencia a que el nivel de ingresos, ya que las diferencias porcentuales entre categorías no son grandes, no se está seguro de las implicancias de la metodología de agrupación utilizada y tampoco siguen la lógica que nos diría que los de menores ingresos son los más propensos a impagos.

Estas conclusiones son para esta muestra. Es posible que en otras muestras se siga el patrón denominado "lógico".

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [89]:
test_hyp_4 = credit_scoring.pivot_table(index='purpose_category', columns='debt', values='dob_years', aggfunc='count', margins=True)
# generando el conteo de valores sobre la categoría de propósitos y tasa de incumplimiento utilizando una tabla dinámica y guardándola
print(test_hyp_4)
# mostrando la tabla pivote
table_hyp_4 = test_hyp_4[1] / test_hyp_4['All']
# generando los porcentajes
table_hyp_4
# comprobando la serie final con las tasas de incumplimiento en función del propósito del préstamo

debt                  0     1    All
purpose_category                    
car purchase       3903   403   4306
education          3643   370   4013
house purchase     3553   256   3809
property           2348   190   2538
real estate        4127   336   4463
wedding            2138   186   2324
All               19712  1741  21453


purpose_category
car purchase      0.093590
education         0.092200
house purchase    0.067209
property          0.074862
real estate       0.075286
wedding           0.080034
All               0.081154
dtype: float64

**Conclusión**

Tomaremos como referencia el porcentaje promedio que es de 8.11 %
* De manera general, notamos que sí existen diferencias relativamente mayores entre categorías y respecto al promedio, exceptuando "wedding", con 8 %.
* La categoría con mayor tasa de incumplimiento es la de "car purchase". Esto indicaría que las personas con ese objetivo de compra tienen ligeramente más probabilidades de tener un impago con un 9.36 %.
* La categoría menos riesgosa en ese sentido sería la de "home purchase", con un 6.72 %. Esto podría tener sentido debido a que se trata de una inversión más cuantiosa y se tienen más precauciones al dar un préstamo con este objetivo.
* Finalmente, afirmamos que los objetivos del préstamo afectan a la tasa de incumplimiento en esta muestra. Las posibles razones son explicadas de manera superficial en líneas previas.

## Conclusión general <a id='conclusion'></a>

Empezaremos con una visión general del preprocesamiento de datos:
* Analizamos el datset y observamos numerosos problemas, los cuales nos planteamos solucionar.
* Luego, transformamos nuestras columnas defectuosas, nos encargamos de los valores extraños y los duplicados.
* A continuación, nos encargamos de los valores ausentes, analizando y generando funciones ciertamente escalables para ayudarnos a solucionarlo.
* También agrupamos nuestras columnas para mejorar la calidad de nuestro análisis, teniendo en cuenta algunos criterios teóricos y del autor.

Una vez solucionado esto, pasamos a probar nuestras hipótesis:
* Pudimos demostrar la primera hipótesis de manera general, ya que existe una brecha entre los que tienen hijos y los que no. Pero no es claro que exista una tendencia acerca del número de hijos cuando es mayor o igual a 1.
* Pudimos demostrar también que existen diferencias ligeras entre la situación familiar y las tasas de impago, confirmando nuestra primera hipótesis. Aunque tenemos ciertos detalles concretos que requerirían un mayor análisis para su total confirmación.
* Para en caso del nivel de ingresos, no se pudo demostrar la hipótesis debido a que las diferencias son más ajustadas, no se tiene claro el alcance de la metodología de agrupación utilizada y tampoco siguen el patrón lógico esperado a totalidad.
* En la hipótesis final, sí se notaron diferencias relativamente mayores entre los diversos propósitos hacia la tasa de impago. Esto nos ayuda a confirmar que esta hipótesis puede ser demostrada. 

Si bien es cierto que las diferencias para cada hipótesis son pequeñas y pueden estar sujetas a subjetividad, se trató de realizarlas justificando la decisión y explicándola. También tengamos en cuenta que mucho de este análisis depende bastante de conocimientos e información de los clientes y la empresa que no se tiene actualmente. Para un entendimiento más acertado de los motivos de ciertas diferencias o similitudes entre categorías se necesitaría analizar las políticas de otorgamiento de créditos y comportamiento de nuestros clientes, así como discusiones del tema con el área respectiva de otorgamiento de créditos. Y nuevamente recordemos que este análisis no fue ciertamente riguroso de manera estadística, ya que no se puede realizar inferencia ni proyecciones estadísticas. Nuestras conclusiones no son generalizables y podrían ser diferentes al comportamiento teórico o empírico determinado en muestras más grandes y con mejores herramientas de análisis.
